<a href="https://colab.research.google.com/github/charlesxjyang/Medium-GPT2/blob/master/GPT_2_Text_Generating_Model_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

Lightly edited by [Charles Yang](http://charlesyang.io/)

*Last updated: November 10th, 2019*
*Last edited: January, 12th, 2020*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [2]:
!nvidia-smi

Sun Jan 12 21:18:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [3]:
gpt2.download_gpt2(model_name="355M")

Fetching checkpoint: 1.05Mit [00:00, 458Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 144Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 422Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:06, 209Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 266Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 110Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 161Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [4]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [0]:
file_name = "all_texts.txt"
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [0]:
run_name = 'run1_355_final'

In [7]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              steps=200,
              restore_from='fresh',
              run_name=run_name,
              print_every=10,
              sample_every=100,
              save_every=100
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:21<00:00, 21.11s/it]


dataset has 4526153 tokens
Training...
[10 | 17.10] loss=3.63 avg=3.63
[20 | 25.94] loss=3.32 avg=3.48
[30 | 34.76] loss=3.23 avg=3.39
[40 | 43.62] loss=2.95 avg=3.28
[50 | 52.46] loss=3.44 avg=3.31
[60 | 61.30] loss=2.98 avg=3.26
[70 | 70.13] loss=2.96 avg=3.21
[80 | 78.95] loss=3.22 avg=3.21
[90 | 87.75] loss=2.86 avg=3.17
[100 | 96.59] loss=2.99 avg=3.15
Saving checkpoint/run1_355_final/model-100
======== SAMPLE 1 ========
 as you get older. The more you do this, the more you become aware of your own strength.
You have found a new and exciting direction, but there is just one obstacle, the one you have to work out of your face every single day. You realize that you don’t have the passion, but you still have the desire, and you’ll be there in about a year or two. Then, it’s on another, perhaps not so good route.
That was one of the reasons why I was so excited in the past: I could see myself in a decade or more as a climber. I wanted more than just riding, I wanted to be the best at 

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name=run_name)

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:

gpt2.copy_checkpoint_from_gdrive(run_name=run_name)

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [9]:
import tensorflow as tf
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=run_name)

Loading checkpoint checkpoint/run1_355_final/model-200
INFO:tensorflow:Restoring parameters from checkpoint/run1_355_final/model-200


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [10]:
gpt2.generate(sess, run_name=run_name)

Product Details

The new Guild Wars 2 is out, and we’ve got a brand new map (complete with an amazing new terrain-and-water-texture) to show you a few hours of the latest action.
This map is from the Guild Wars 2: Rise of Iron DLC, and it’s pack-in-there-for-the-heroes-of-Iron-DLC-set. You can check it out here.
Now, we’re not going to cover the maps and terrain, but you can check out the video below, which shows you the build-in-for-heroes.
The map is not an official preview, but it’s worth a look.
From the get-go, we’ve decided to focus on terrainy maps, with the newest terrain-and-water-texture. More on that soon.
As you can see, this map is already packed-in for the heroes, which means we’re not going to cover the maps and terrain, but instead focus on the maps and terrain.
The map is built on Guild Wars 2’s Guild Wars 2: Rise of Iron DLC pack. It contains three maps, and we’ve included two of the maps, Water and Ice.
This map contains two maps, Water and Ice. Water features a new 

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [13]:
gpt2.generate(sess,
              run_name=run_name,
              length=100,
              temperature=0.7,
              prefix="Data science is ",
              nsamples=15,
              batch_size=1
              )

Data science is ・“data”. Data science is data, data, data. Data science is data, data, data. Data science is data, data, data. Data science is data, data, data. Data science is data, data, data. Data science is data, data, data. Data science is data, data, data. Data science is data, data, data. Data science is data, data, data. Data science is data, data, data. Data science is
Data science is  a tool that allows us to value data in a way that is easier to understand and use.
"
"Dataset:
For this dataset you can use the Dataset command
I used the Dataset command to collect data on the amount of energy used by the solar panels. Dataset is a great tool for data analysis. You can see my code in the Dataset command.
When you see the data, make sure to click the green check mark
Data science is _________________
"
"This is a guest blog post by Karen Wallace, a writer for DataCamp.org.
The last time I used the term data science in my blog post was April 20, 2017. There are many different way

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [0]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [3]:
model_name = "1558M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 302Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 104Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 792Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 6.23Git [02:06, 49.3Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 477Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 170Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 169Mit/s]                                                       


In [4]:
import tensorflow as tf
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, model_name=model_name)

Loading pretrained model models/1558M/model.ckpt
INFO:tensorflow:Restoring parameters from models/1558M/model.ckpt


In [0]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="Artificial General Intelligence will arrive",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=10,
              batch_size=5
              )

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.